In [1]:
import numpy as np
import ipywidgets as widgets
from IPython.display import Markdown

In [205]:
class MarsRover:
    def __init__(self, transition_probabilities=np.ones((5,2)), rewards=[1, 0, 0, 0, 10], horizon=10):
        self.rewards = rewards
        self.probs = transition_probabilities
        self.c_steps = 0
        self.horizon = horizon

    def reset(self):
        self.c_steps = 0
        self.position = 2
        return self.position

    def step(self, action):
        done = False
        self.c_steps += 1
        follow_action = np.random.choice([0, 1], p=[1-self.probs[self.position][action],self.probs[self.position][action]])
        if not follow_action:
            action = 1 - action

        if action == 0:
            if self.position > 0:
                self.position -= 1
        elif action == 1:
            if self.position < 4:
                self.position += 1
        else:
            print("Not a valid action")
            return
        reward = self.rewards[self.position]
        return self.position, reward, self.c_steps >= self.horizon

In [265]:
def dynamic_programming_step(v, pi, transition_probabilities, rewards, gamma=0.9):
    new_v = np.copy(v)
    for s in range(5):
        action = pi[s]
        next_state = min(4, max(0, s + action + (action-1)))
        alternate_state = min(4, max(0, s - action + np.abs(action-1)))
        new_v[s] = rewards[next_state] + gamma * (transition_probabilities[s][action]*v[next_state]+(1-transition_probabilities[s][action])*v[alternate_state])
    return new_v

In [207]:
def monte_carlo_step(v, first_visits, total_returns, sample, gamma=0.9):
    v_new = np.copy(v)
    updated_visits = np.copy(first_visits)
    updated_returns = np.copy(total_returns)
    visited_this_episode = np.zeros(5)
    for i in range(len(sample)):
        if i%3 == 0 and not visited_this_episode[sample[i]]:
            updated_visits[sample[i]] += 1
            future_rewards = [sample[j] for j in range(i+2, len(sample), 3)]
            acc_future_rewards = 0
            for k in range(len(future_rewards)):
                acc_future_rewards += (gamma ** k) * future_rewards[k] 
            updated_returns[sample[i]] += acc_future_rewards
            v_new[sample[i]] = updated_returns[sample[i]] / updated_visits[sample[i]]
            visited_this_episode[sample[i]] = 1
    return v_new, updated_visits, updated_returns

In [208]:
def td_zero_step(v, sample, alpha=0.1, gamma=0.9):
    v_new = np.copy(v)
    state = sample[0]
    reward = sample[2]
    next_state = sample[3]
    
    v_new[state] = v[state] + alpha * (reward + gamma * v[next_state] - v[state])
    return v_new

In [216]:
pi = np.random.randint(2, size=5)

In [217]:
transition_probabilities=np.ones((5, 2))
rewards=[1, 0, 0, 0, 10]
env = MarsRover(transition_probabilities=transition_probabilities, rewards=rewards)

mars_rover_samples = []
mars_rover_episodes = []
for _ in range(500):
    state = env.reset()
    done = False
    ep = [state]
    while not done:
        action = pi[state]
        next_state, reward, done = env.step(action)
        mars_rover_samples.append([state, action, reward, next_state])
        ep.append(action)
        ep.append(reward)
        ep.append(next_state)
        state = next_state
    mars_rover_episodes.append(ep)
np.random.shuffle(mars_rover_samples)
np.random.shuffle(mars_rover_episodes)

In [218]:
state = env.reset()
v_dp = np.zeros(5)

In [224]:
v_dp = dynamic_programming_step(v_dp, pi, transition_probabilities, rewards)
table = Markdown("""
## The value function in dynamic programming
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by DP    |   {v_dp[0]}  |    {v_dp[1]}     |   {v_dp[2]}         |     {v_dp[3]}       | {v_dp[4]}      |
""".format(v_dp=v_dp))
display(table)
display(Markdown("""## For policy {pi}""".format(pi=pi)))


## The value function in dynamic programming
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by DP    |   43.3755  |    46.153800000000004     |   96.47999999999999         |     102.38050000000001       | 124.26299999999999      |


## For policy [1 1 1 1 0]

In [239]:
state = env.reset()
v_mc = np.zeros(5)
first_visits = np.zeros(5)
total_returns = np.zeros(5)

In [242]:
random_index = np.random.randint(len(mars_rover_episodes))
sample_episode = mars_rover_episodes[random_index]
display(Markdown(f"""## Sample episode used to update: {sample_episode}"""))
v_mc, first_visits, total_returns = monte_carlo_step(v_mc, first_visits, total_returns, sample_episode)
table = Markdown("""
## The value function in MC
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by MC    |   {v_mc[0]}  |    {v_mc[1]}     |   {v_mc[2]}         |     {v_mc[3]}       | {v_mc[4]}      |
""".format(v_mc=v_mc))
display(table)
display(Markdown("""## For policy {pi}""".format(pi=pi)))

## Sample episode used to update: [2, 1, 0, 3, 1, 10, 4, 0, 0, 3, 1, 10, 4, 0, 0, 3, 1, 10, 4, 0, 0, 3, 1, 10, 4, 0, 0, 3, 1, 10, 4]


## The value function in MC
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by MC    |   0.0  |    0.0     |   30.852073890000003         |     34.2800821       | 26.977869000000002      |


## For policy [1 1 1 1 0]

In [232]:
state = env.reset()
v_td = np.zeros(5)

In [238]:
random_index = np.random.randint(len(mars_rover_samples))
sample_transition = mars_rover_samples[random_index]
display(Markdown(f"""## Sample transition used to update: {sample_transition}"""))
v_td = td_zero_step(v_td, sample_transition)
table = Markdown("""
## The value function in TD(0)
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by TD(0) |   {v_td[0]}  |    {v_td[1]}     |   {v_td[2]}         |     {v_td[3]}       | {v_td[4]}      |
""".format(v_td=v_td))
display(table)
display(Markdown("""## For policy {pi}""".format(pi=pi)))

## Sample transition used to update: [3, 1, 10, 4]


## The value function in TD(0)
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by TD(0) |   0.0  |    0.0     |   0.3249         |     2.72539       | 0.171      |


## For policy [1 1 1 1 0]

In [266]:
state = env.reset()

v_dp = np.zeros(5)
v_mc = np.zeros(5)
first_visits = np.zeros(5)
total_returns = np.zeros(5)
v_td = np.zeros(5)

In [267]:
display(Markdown("""# Comparing value function development for policy {pi}""".format(pi=pi)))

# Comparing value function development for policy [1 1 1 1 0]

In [277]:
v_dp = dynamic_programming_step(v_dp, pi, transition_probabilities, rewards)

random_index = np.random.randint(len(mars_rover_episodes))
sample_episode = mars_rover_episodes[random_index]
v_mc, first_visits, total_returns = monte_carlo_step(v_mc, first_visits, total_returns, sample_episode)

for _ in range(10):
    random_index = np.random.randint(len(mars_rover_samples))
    sample_transition = mars_rover_samples[random_index]
    v_td = td_zero_step(v_td, sample_transition)

table = Markdown("""
## Comparing value function development
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by DP    |   {v_dp[0]}  |    {v_dp[1]}     |   {v_dp[2]}         |     {v_dp[3]}       | {v_dp[4]}      |
|    V by MC    |   {v_mc[0]}  |    {v_mc[1]}     |   {v_mc[2]}         |     {v_mc[3]}       | {v_mc[4]}      |
|    V by TD(0) |   {v_td[0]}  |    {v_td[1]}     |   {v_td[2]}         |     {v_td[3]}       | {v_td[4]}      |
""".format(v_dp=v_dp, v_mc=v_mc, v_td=v_td))
display(table)


## Comparing value function development
|               | Good view | Nothing interesting | Nothing interesting | Nothing interesting | Very important science |
|:-------------:|:---------:|:-------------------:|:-------------------:|:-------------------:|:----------------------:| 
|    V by DP    |   21.852073890000003  |    24.2800821     |   30.852073890000003         |     34.2800821       | 30.852073890000003      |
|    V by MC    |   0.0  |    0.0     |   30.852073890000003         |     34.28008210000001       | 26.977869000000005      |
|    V by TD(0) |   0.0  |    0.0     |   7.326740023744119         |     21.32720889404298       | 15.57925213411243      |
